In [ ]:
!pip install torchvision

In [ ]:
import copy
import pathlib

import gymnasium as gym
import numpy as np
import torch
import torchvision
from lightning.fabric import Fabric
from omegaconf import OmegaConf
from PIL import Image

from sheeprl.algos.dreamer_v3.agent import PlayerDV3, build_agent
from sheeprl.data.buffers import SequentialReplayBuffer
from sheeprl.utils.env import make_env
from sheeprl.utils.utils import dotdict

In [ ]:
# path of your checkpoint
ckpt_path = pathlib.Path("/path/to/your/checkpoint.ckpt")

In [ ]:
seed = 5
fabric = Fabric(accelerator="cuda", devices=1)
state = fabric.load(ckpt_path)
cfg = dotdict(OmegaConf.to_container(OmegaConf.load(ckpt_path.parent.parent / "config.yaml"), resolve=True))
cfg.env.num_envs = 1

In [ ]:
envs = gym.vector.AsyncVectorEnv(
    [
        make_env(
            cfg,
            cfg.seed + 0 * cfg.env.num_envs + i,
            0 * cfg.env.num_envs,
            "./imagination",
            "imagination",
            vector_env_idx=i,
        )
        for i in range(cfg.env.num_envs)
    ]
)
action_space = envs.single_action_space
observation_space = envs.single_observation_space
obs_keys = cfg.algo.cnn_keys.encoder + cfg.algo.mlp_keys.encoder

In [ ]:
is_continuous = isinstance(action_space, gym.spaces.Box)
is_multidiscrete = isinstance(action_space, gym.spaces.MultiDiscrete)
actions_dim = tuple(
    action_space.shape if is_continuous else (action_space.nvec.tolist() if is_multidiscrete else [action_space.n])
)
world_model, actor, critic, critic_target = build_agent(
    fabric,
    actions_dim,
    is_continuous,
    cfg,
    observation_space,
    state["world_model"],
    state["actor"],
    state["critic"],
    state["target_critic"],
)
player = PlayerDV3(
    world_model.encoder.module,
    world_model.rssm,
    actor.module,
    actions_dim,
    cfg.env.num_envs,
    cfg.algo.world_model.stochastic_size,
    cfg.algo.world_model.recurrent_model.recurrent_state_size,
    fabric.device,
    cfg.algo.world_model.discrete_size,
)

In [ ]:
initial_steps = 200  # set according to your environment
imagination_steps = 45  # number of imagination steps
clip_rewards_fn = lambda r: torch.tanh(r) if cfg.env.clip_rewards else r
rb_imagination = SequentialReplayBuffer(imagination_steps, cfg.env.num_envs)
rb_play = SequentialReplayBuffer(imagination_steps, cfg.env.num_envs)
rb_initial = SequentialReplayBuffer(initial_steps, cfg.env.num_envs)
step_data = {}
player.init_states()
obs = envs.reset(seed=cfg.seed)[0]
for k in obs_keys:
    step_data[k] = obs[k][np.newaxis]
step_data["dones"] = np.zeros((1, cfg.env.num_envs, 1))
step_data["rewards"] = np.zeros((1, cfg.env.num_envs, 1))
step_data["is_first"] = np.ones_like(step_data["dones"])
step_data["stochastic_state"] = player.stochastic_state.detach().cpu().numpy()
step_data["recurrent_state"] = player.recurrent_state.detach().cpu().numpy()

In [ ]:
info_list = []
# play for `initial_steps` steps
for i in range(initial_steps):
    with torch.no_grad():
        preprocessed_obs = {}
        for k, v in obs.items():
            preprocessed_obs[k] = torch.as_tensor(v[np.newaxis], dtype=torch.float32, device=fabric.device)
            if k in cfg.algo.cnn_keys.encoder:
                preprocessed_obs[k] = preprocessed_obs[k] / 255.0
        mask = {k: v for k, v in preprocessed_obs.items() if k.startswith("mask")}
        if len(mask) == 0:
            mask = None
        real_actions = actions = player.get_exploration_action(preprocessed_obs, mask)
        actions = torch.cat(actions, -1).cpu().numpy()
        if is_continuous:
            real_actions = torch.cat(real_actions, dim=-1).cpu().numpy()
        else:
            real_actions = torch.cat([real_act.argmax(dim=-1) for real_act in real_actions], dim=-1).cpu().numpy()

    step_data["actions"] = actions.reshape((1, cfg.env.num_envs, -1))
    rb_initial.add(step_data, validate_args=cfg.buffer.validate_args)

    next_obs, rewards, dones, truncated, infos = envs.step(real_actions.reshape(envs.action_space.shape))
    rewards = np.array(rewards).reshape((1, cfg.env.num_envs, -1))
    dones = np.logical_or(dones, truncated).astype(np.uint8).reshape((1, cfg.env.num_envs, -1))

    step_data["is_first"] = np.zeros_like(step_data["dones"])
    if "restart_on_exception" in infos:
        for i, agent_roe in enumerate(infos["restart_on_exception"]):
            if agent_roe and not dones[i]:
                last_inserted_idx = (rb_initial.buffer[i]._pos - 1) % rb_initial.buffer[i].buffer_size
                rb_initial.buffer[i]["dones"][last_inserted_idx] = np.ones_like(
                    rb_initial.buffer[i]["dones"][last_inserted_idx]
                )
                rb_initial.buffer[i]["is_first"][last_inserted_idx] = np.zeros_like(
                    rb_initial.buffer[i]["is_first"][last_inserted_idx]
                )
                step_data["is_first"][i] = np.ones_like(step_data["is_first"][i])

    real_next_obs = copy.deepcopy(next_obs)
    if "final_observation" in infos:
        for idx, final_obs in enumerate(infos["final_observation"]):
            if final_obs is not None:
                for k, v in final_obs.items():
                    real_next_obs[k][idx] = v

    for k in obs_keys:
        step_data[k] = next_obs[k][np.newaxis]

    obs = next_obs

    rewards = rewards.reshape((1, cfg.env.num_envs, -1))
    step_data["dones"] = dones.reshape((1, cfg.env.num_envs, -1))
    step_data["rewards"] = clip_rewards_fn(rewards)
    step_data["rewards"] = clip_rewards_fn(rewards)
    step_data["stochastic_state"] = player.stochastic_state.detach().cpu().numpy()
    step_data["recurrent_state"] = player.recurrent_state.detach().cpu().numpy()
    dones_idxes = dones.nonzero()[0].tolist()
    reset_envs = len(dones_idxes)
    if reset_envs > 0:
        reset_data = {}
        for k in obs_keys:
            reset_data[k] = (real_next_obs[k][dones_idxes])[np.newaxis]
        reset_data["dones"] = np.ones((1, reset_envs, 1))
        reset_data["actions"] = np.zeros((1, reset_envs, np.sum(actions_dim)))
        reset_data["rewards"] = step_data["rewards"][:, dones_idxes]
        reset_data["is_first"] = np.zeros_like(reset_data["dones"])
        rb_initial.add(reset_data, dones_idxes, validate_args=cfg.buffer.validate_args)

        # Reset already inserted step data
        step_data["rewards"][:, dones_idxes] = np.zeros_like(reset_data["rewards"])
        step_data["dones"][:, dones_idxes] = np.zeros_like(step_data["dones"][:, dones_idxes])
        step_data["is_first"][:, dones_idxes] = np.ones_like(step_data["is_first"][:, dones_idxes])
        player.init_states(dones_idxes)
    if i == initial_steps - imagination_steps - 1:
        stochastic_state = player.stochastic_state.clone()
        recurrent_state = player.recurrent_state.clone()

In [ ]:
# imagination / reconstruction obs process
imagined_latent_states = torch.cat((stochastic_state, recurrent_state), -1)
step_data = {}
reconstruced_step_data = {}
with torch.no_grad():
    for i in range(imagination_steps):
        # deciede if you want to take the actions from the buffer
        # (i.e., the actions actually played by the agent)
        # or imagine them and compare with the actions actually played by the agent

        # actions actually played by the agent
        # actions = torch.tensor(
        #     rb_initial["actions"][-imagination_steps + i],
        #     device=fabric.device,
        #     dtype=torch.float32,
        # )[None]

        # imagined actions
        actions = actor(imagined_latent_states.detach())[0][0]

        # imagination step
        imagined_stochastic_state, recurrent_state = world_model.rssm.imagination(
            stochastic_state, recurrent_state, actions
        )
        # update current state
        imagined_latent_states = torch.cat((imagined_stochastic_state.view(1, 1, -1), recurrent_state), -1)
        rec_obs = world_model.observation_model(imagined_latent_states)
        step_data["rgb"] = rec_obs["rgb"].unsqueeze(0).detach().cpu().numpy()
        step_data["actions"] = actions.unsqueeze(0).detach().cpu().numpy()
        rb_imagination.add(step_data)

        # reconstruct the observations from the latent states used when interacting with the environment
        played_latent_states = torch.cat(
            (
                torch.tensor(rb_initial["stochastic_state"][-imagination_steps + i], device=fabric.device),
                torch.tensor(rb_initial["recurrent_state"][-imagination_steps + i], device=fabric.device),
            ),
            -1,
        )
        rec_obs_played = world_model.observation_model(played_latent_states)
        reconstruced_step_data["rgb"] = rec_obs_played["rgb"].unsqueeze(0).detach().cpu().numpy()
        rb_play.add(reconstruced_step_data)

In [ ]:
# create a gif with the imagined frames (the latent state is computed by the transition model)
frames = torch.clamp(torch.tensor(rb_imagination["rgb"][:, 0, 0]), 0, 1).detach()
frames = [torchvision.transforms.functional.to_pil_image(f) for f in frames]
frame_one = frames[0]
frame_one.save("imagination.gif", format="GIF", append_images=frames, save_all=True, duration=100, loop=0)

In [ ]:
# create a gif with the reconstructed observations (the latent state is computed by the representation model)
frames = torch.clamp(torch.tensor(rb_play["rgb"][:, 0]), 0, 1).detach()
frames = [torchvision.transforms.functional.to_pil_image(f) for f in frames]
frame_one = frames[0]
frame_one.save("reconstructed_obs.gif", format="GIF", append_images=frames, save_all=True, duration=100, loop=0)

In [ ]:
# create a gif with the real observations (provided by the environment)
frames = torch.tensor(rb_initial["rgb"][-imagination_steps:, 0])
frames = [torchvision.transforms.functional.to_pil_image(f) for f in frames]
frame_one = frames[0]
frame_one.save("real_obs.gif", format="GIF", append_images=frames, save_all=True, duration=100, loop=0)